# Monte Carlo Portfolio Simulation
## Multi-Asset | Regime-Switching | Fat-Tailed | Risk Parity

This notebook implements a comprehensive Monte Carlo simulation for a multi-asset portfolio with the following features:

- **8 assets** across Equities, Bonds, Commodities, and FX
- **Correlated fat-tailed returns** via Cholesky decomposition with Student-t innovations
- **3-state Markov regime switching** (Bull / Bear / Crisis) with distinct parameters per regime
- **Risk parity allocation** — equal-risk-contribution optimization
- **Macro factor decomposition** into Growth, Inflation, and Rate drivers
- **Stress testing** against GFC 2008, COVID 2020, Rate Shock, and Stagflation scenarios
- **Comprehensive risk metrics**: VaR, CVaR, Sharpe, Sortino, Calmar, Maximum Drawdown

---

## 1. Setup & Configuration

We define the full asset universe, simulation parameters, regime-specific return distributions, Markov transition probabilities, macro factor loadings, and stress scenarios.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize
import time
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('dark_background')
plt.rcParams['figure.dpi'] = 120

print("Libraries loaded successfully.")

In [ ]:
# ─── Asset Universe ───────────────────────────────────────────────────────────

ASSETS = [
    "US_Equity", "Intl_Equity", "US_Treasury_10Y", "TIPS",
    "Gold", "Commodities", "USD_EUR", "USD_JPY",
]

ASSET_CLASSES = {
    "US_Equity": "Equity", "Intl_Equity": "Equity",
    "US_Treasury_10Y": "Bond", "TIPS": "Bond",
    "Gold": "Commodity", "Commodities": "Commodity",
    "USD_EUR": "FX", "USD_JPY": "FX",
}

N_ASSETS = len(ASSETS)

# ─── Simulation Parameters ────────────────────────────────────────────────────

N_SIMULATIONS = 10_000
N_YEARS = 10
TRADING_DAYS_PER_YEAR = 252
N_PERIODS = TRADING_DAYS_PER_YEAR * N_YEARS  # 2520
DT = 1.0 / TRADING_DAYS_PER_YEAR
RISK_FREE_RATE = 0.04  # annualized
SEED = 42

print(f"Asset Universe: {N_ASSETS} assets across {len(set(ASSET_CLASSES.values()))} classes")
print(f"Simulation: {N_SIMULATIONS:,} paths x {N_PERIODS:,} days ({N_YEARS} years)")
print(f"Risk-free rate: {RISK_FREE_RATE:.1%}")

### Regime Parameters

Each market regime has distinct characteristics:

| Regime | Expected Returns | Volatility | Correlations | Tail Thickness (df) |
|--------|-----------------|------------|--------------|--------------------|
| **Bull** | Positive | Low | Moderate | 8 (mild tails) |
| **Bear** | Negative/Low | High | Elevated | 5 (moderate tails) |
| **Crisis** | Large negative | Very high | Spike toward 1 | 3 (extreme tails) |

The correlation spike during crises captures the empirical phenomenon where diversification fails precisely when it's needed most.

In [ ]:
# ─── Regime Parameters ────────────────────────────────────────────────────────

# Bull regime — moderate returns, low vol, moderate correlations
BULL_MU = np.array([0.10, 0.08, 0.03, 0.04, 0.05, 0.06, 0.01, -0.01])
BULL_SIGMA = np.array([0.15, 0.17, 0.06, 0.07, 0.16, 0.20, 0.08, 0.10])
BULL_CORR = np.array([
    [ 1.00,  0.80, -0.20, -0.10,  0.05,  0.30,  0.10,  0.05],
    [ 0.80,  1.00, -0.15, -0.05,  0.10,  0.35,  0.25,  0.15],
    [-0.20, -0.15,  1.00,  0.70, -0.05, -0.10, -0.05,  0.10],
    [-0.10, -0.05,  0.70,  1.00,  0.15,  0.10,  0.00,  0.05],
    [ 0.05,  0.10, -0.05,  0.15,  1.00,  0.40,  0.20,  0.10],
    [ 0.30,  0.35, -0.10,  0.10,  0.40,  1.00,  0.15,  0.05],
    [ 0.10,  0.25, -0.05,  0.00,  0.20,  0.15,  1.00,  0.30],
    [ 0.05,  0.15,  0.10,  0.05,  0.10,  0.05,  0.30,  1.00],
])
BULL_DF = 8

# Bear regime — negative/low returns, high vol, higher correlations
BEAR_MU = np.array([-0.05, -0.08, 0.06, 0.03, 0.08, -0.03, 0.02, 0.03])
BEAR_SIGMA = np.array([0.25, 0.28, 0.10, 0.12, 0.22, 0.30, 0.12, 0.14])
BEAR_CORR = np.array([
    [ 1.00,  0.90, -0.30, -0.15,  0.15,  0.50,  0.20,  0.10],
    [ 0.90,  1.00, -0.25, -0.10,  0.20,  0.55,  0.35,  0.20],
    [-0.30, -0.25,  1.00,  0.75, -0.10, -0.20, -0.10,  0.15],
    [-0.15, -0.10,  0.75,  1.00,  0.20,  0.05,  0.00,  0.10],
    [ 0.15,  0.20, -0.10,  0.20,  1.00,  0.45,  0.25,  0.15],
    [ 0.50,  0.55, -0.20,  0.05,  0.45,  1.00,  0.20,  0.10],
    [ 0.20,  0.35, -0.10,  0.00,  0.25,  0.20,  1.00,  0.40],
    [ 0.10,  0.20,  0.15,  0.10,  0.15,  0.10,  0.40,  1.00],
])
BEAR_DF = 5

# Crisis regime — large drawdowns, extreme vol, correlations spike
CRISIS_MU = np.array([-0.30, -0.35, 0.10, 0.02, 0.15, -0.20, 0.05, 0.08])
CRISIS_SIGMA = np.array([0.45, 0.50, 0.15, 0.18, 0.30, 0.45, 0.18, 0.20])
CRISIS_CORR = np.array([
    [ 1.00,  0.95, -0.40, -0.20,  0.25,  0.70,  0.30,  0.15],
    [ 0.95,  1.00, -0.35, -0.15,  0.30,  0.75,  0.45,  0.25],
    [-0.40, -0.35,  1.00,  0.80, -0.15, -0.30, -0.15,  0.20],
    [-0.20, -0.15,  0.80,  1.00,  0.25,  0.00,  0.00,  0.15],
    [ 0.25,  0.30, -0.15,  0.25,  1.00,  0.50,  0.30,  0.20],
    [ 0.70,  0.75, -0.30,  0.00,  0.50,  1.00,  0.25,  0.15],
    [ 0.30,  0.45, -0.15,  0.00,  0.30,  0.25,  1.00,  0.50],
    [ 0.15,  0.25,  0.20,  0.15,  0.20,  0.15,  0.50,  1.00],
])
CRISIS_DF = 3

REGIMES = {
    "bull":   {"mu": BULL_MU,   "sigma": BULL_SIGMA,   "corr": BULL_CORR,   "df": BULL_DF},
    "bear":   {"mu": BEAR_MU,   "sigma": BEAR_SIGMA,   "corr": BEAR_CORR,   "df": BEAR_DF},
    "crisis": {"mu": CRISIS_MU, "sigma": CRISIS_SIGMA, "corr": CRISIS_CORR, "df": CRISIS_DF},
}
REGIME_NAMES = ["bull", "bear", "crisis"]

# ─── Markov Transition Matrix ─────────────────────────────────────────────────
TRANSITION_MATRIX = np.array([
    [0.980, 0.015, 0.005],  # bull  → bull 98%, bear 1.5%, crisis 0.5%
    [0.030, 0.950, 0.020],  # bear  → bull 3%,  bear 95%,  crisis 2%
    [0.050, 0.150, 0.800],  # crisis→ bull 5%,  bear 15%,  crisis 80%
])

print("Regime parameters configured.")
print(f"Transition matrix row sums: {TRANSITION_MATRIX.sum(axis=1)}")

In [ ]:
# ─── Macro Factor Loadings ────────────────────────────────────────────────────
# 8 assets x 3 factors: [Growth, Inflation, Rates]
#   Growth:    equities +, bonds -, commodities +
#   Inflation: TIPS +, gold +, commodities +, nominal bonds -
#   Rates:     bonds -, equities -, TIPS partial hedge

FACTOR_LOADINGS = np.array([
    [ 0.80,  0.10, -0.20],  # US_Equity
    [ 0.75,  0.15, -0.15],  # Intl_Equity
    [-0.30, -0.40,  0.70],  # US_Treasury_10Y
    [-0.10,  0.50,  0.30],  # TIPS
    [ 0.10,  0.60, -0.10],  # Gold
    [ 0.40,  0.55, -0.05],  # Commodities
    [ 0.20,  0.05, -0.15],  # USD_EUR
    [-0.10, -0.05,  0.25],  # USD_JPY
])

FACTOR_NAMES = ["Growth", "Inflation", "Rates"]

# ─── Stress Scenarios ─────────────────────────────────────────────────────────
STRESS_SCENARIOS = {
    "GFC 2008": {
        "US_Equity": -0.38, "Intl_Equity": -0.43, "US_Treasury_10Y": 0.20,
        "TIPS": 0.02, "Gold": 0.05, "Commodities": -0.36,
        "USD_EUR": -0.04, "USD_JPY": 0.23,
    },
    "COVID 2020": {
        "US_Equity": -0.34, "Intl_Equity": -0.30, "US_Treasury_10Y": 0.15,
        "TIPS": 0.05, "Gold": 0.03, "Commodities": -0.25,
        "USD_EUR": -0.02, "USD_JPY": 0.03,
    },
    "Rate Shock": {
        "US_Equity": -0.15, "Intl_Equity": -0.12, "US_Treasury_10Y": -0.18,
        "TIPS": -0.08, "Gold": -0.05, "Commodities": 0.05,
        "USD_EUR": -0.05, "USD_JPY": -0.08,
    },
    "Stagflation": {
        "US_Equity": -0.20, "Intl_Equity": -0.22, "US_Treasury_10Y": -0.10,
        "TIPS": 0.05, "Gold": 0.15, "Commodities": 0.25,
        "USD_EUR": 0.03, "USD_JPY": -0.05,
    },
}

print(f"Factor loadings: {FACTOR_LOADINGS.shape[0]} assets x {FACTOR_LOADINGS.shape[1]} factors")
print(f"Stress scenarios: {list(STRESS_SCENARIOS.keys())}")

## 2. Markov Regime-Switching Model

The regime path is generated via a discrete-time Markov chain. At each time step, the market transitions between Bull, Bear, and Crisis states according to the transition matrix. The stationary distribution gives us the long-run probability of being in each regime.

$$\pi P = \pi, \quad \sum_i \pi_i = 1$$

In [ ]:
def stationary_distribution(transition_matrix):
    """Compute the stationary distribution of a Markov chain."""
    n = transition_matrix.shape[0]
    A = transition_matrix.T - np.eye(n)
    A[-1, :] = 1.0
    b = np.zeros(n)
    b[-1] = 1.0
    return np.linalg.solve(A, b)


def simulate_regime_paths(transition_matrix, n_sims, n_periods, rng):
    """
    Simulate Markov chain regime paths for all simulations.
    Returns array of shape (n_sims, n_periods) with labels 0=bull, 1=bear, 2=crisis.
    """
    n_regimes = transition_matrix.shape[0]
    pi = stationary_distribution(transition_matrix)
    cum_trans = np.cumsum(transition_matrix, axis=1)

    regimes = np.empty((n_sims, n_periods), dtype=np.int32)
    regimes[:, 0] = rng.choice(n_regimes, size=n_sims, p=pi)

    for t in range(1, n_periods):
        u = rng.random(n_sims)
        current = regimes[:, t - 1]
        cum_probs = cum_trans[current]
        regimes[:, t] = (u[:, None] >= cum_probs).sum(axis=1)

    return regimes


# Show stationary distribution
pi = stationary_distribution(TRANSITION_MATRIX)
print("Stationary Distribution:")
for name, prob in zip(REGIME_NAMES, pi):
    print(f"  {name.capitalize():<8} {prob:.2%}")

## 3. Monte Carlo Simulation Engine

The core engine generates correlated, fat-tailed returns using:

1. **Independent Student-t draws** — captures fat tails (unlike Gaussian)
2. **Variance normalization** — Student-t with $\nu$ df has variance $\nu/(\nu-2)$, so we scale by $\sqrt{(\nu-2)/\nu}$
3. **Cholesky decomposition** — induces the correct correlation structure: $r_t = \mu_{\Delta t} + L \cdot z_t$

Each regime has pre-computed Cholesky factors, so we avoid recomputing them at every time step.

In [ ]:
def is_positive_definite(A):
    try:
        np.linalg.cholesky(A)
        return True
    except np.linalg.LinAlgError:
        return False


def nearest_positive_definite(A):
    """Find the nearest positive-definite matrix (Higham algorithm)."""
    B = (A + A.T) / 2
    _, s, V = np.linalg.svd(B)
    H = V.T @ np.diag(s) @ V
    A2 = (B + H) / 2
    A3 = (A2 + A2.T) / 2
    if is_positive_definite(A3):
        return A3
    spacing = np.spacing(np.linalg.norm(A))
    I = np.eye(A.shape[0])
    k = 1
    while not is_positive_definite(A3):
        mineig = np.min(np.real(np.linalg.eigvalsh(A3)))
        A3 += I * (-mineig * k**2 + spacing)
        k += 1
    return A3


def precompute_regime_params():
    """Pre-compute Cholesky factors and scaled parameters for each regime."""
    params = {}
    for name in REGIME_NAMES:
        r = REGIMES[name]
        mu, sigma, corr, df = r["mu"], r["sigma"], r["corr"], r["df"]
        cov_annual = np.diag(sigma) @ corr @ np.diag(sigma)
        cov_daily = cov_annual * DT
        if not is_positive_definite(cov_daily):
            cov_daily = nearest_positive_definite(cov_daily)
        L = np.linalg.cholesky(cov_daily)
        mu_daily = mu * DT
        t_scale = np.sqrt((df - 2) / df) if df > 2 else 1.0
        params[name] = {"mu_daily": mu_daily, "L": L, "df": df, "t_scale": t_scale, "cov_annual": cov_annual}
    return params


def run_monte_carlo(n_sims=N_SIMULATIONS, seed=SEED):
    """
    Run the full Monte Carlo simulation.
    Returns dict with 'returns', 'prices', 'regimes' arrays.
    """
    rng = np.random.default_rng(seed)
    regime_params = precompute_regime_params()
    regimes = simulate_regime_paths(TRANSITION_MATRIX, n_sims, N_PERIODS, rng)
    returns = np.empty((n_sims, N_PERIODS, N_ASSETS))

    for regime_idx, regime_name in enumerate(REGIME_NAMES):
        p = regime_params[regime_name]
        mask = regimes == regime_idx
        n_draws = mask.sum()
        if n_draws == 0:
            continue
        z = stats.t.rvs(df=p["df"], size=(n_draws, N_ASSETS), random_state=rng)
        z *= p["t_scale"]
        correlated = z @ p["L"].T + p["mu_daily"]
        returns[mask] = correlated

    prices = np.ones((n_sims, N_PERIODS + 1, N_ASSETS))
    prices[:, 1:, :] = np.cumprod(1.0 + returns, axis=1)

    return {"returns": returns, "prices": prices, "regimes": regimes}


print("Simulation engine ready.")

### Run the Simulation

In [ ]:
print(f"Running {N_SIMULATIONS:,} Monte Carlo simulations...")
t0 = time.time()
results = run_monte_carlo(n_sims=N_SIMULATIONS, seed=SEED)
t1 = time.time()
print(f"Completed in {t1 - t0:.1f}s")
print(f"Returns shape: {results['returns'].shape}  (sims x days x assets)")
print(f"Prices shape:  {results['prices'].shape}")
print(f"Regimes shape: {results['regimes'].shape}")

## 4. Covariance Estimation & Expected Returns

We estimate the covariance matrix and expected returns from the simulated data. A subsample of 500,000 observations is used for covariance estimation to keep computation efficient.

In [ ]:
# Annualized expected returns from simulation
mean_daily_returns = results["returns"].mean(axis=(0, 1))
mu_annual = mean_daily_returns * TRADING_DAYS_PER_YEAR

# Covariance estimation (subsample for efficiency)
flat_returns = results["returns"].reshape(-1, N_ASSETS)
rng = np.random.default_rng(SEED)
subsample_idx = rng.choice(flat_returns.shape[0], size=min(500_000, flat_returns.shape[0]), replace=False)
cov_daily = np.cov(flat_returns[subsample_idx], rowvar=False)
cov_annual = cov_daily * TRADING_DAYS_PER_YEAR

print("Expected Annualized Returns:")
for name, ret in zip(ASSETS, mu_annual):
    print(f"  {name:<20} {ret:>8.2%}")

print(f"\nAnnualized volatilities:")
for name, vol in zip(ASSETS, np.sqrt(np.diag(cov_annual))):
    print(f"  {name:<20} {vol:>8.2%}")

## 5. Risk Parity Allocation

Risk parity sets portfolio weights such that each asset contributes equally to total portfolio risk. For weights $w$ and covariance matrix $\Sigma$:

$$RC_i = \frac{w_i \cdot (\Sigma w)_i}{\sqrt{w^T \Sigma w}}$$

We minimize $\sum_i (RC_i - \sigma_p/n)^2$ subject to $\sum w_i = 1$ and $w_i \geq 0.01$.

The initial guess uses inverse-volatility weights, which is already a good approximation.

In [ ]:
def risk_contribution(weights, cov):
    """Compute each asset's percentage contribution to total portfolio risk."""
    port_var = weights @ cov @ weights
    port_vol = np.sqrt(port_var)
    marginal = cov @ weights
    rc = weights * marginal / port_vol
    return rc / rc.sum()


def risk_parity_weights(cov):
    """Compute risk parity weights via optimization."""
    n = cov.shape[0]
    target_rc = np.ones(n) / n

    def objective(w):
        port_var = w @ cov @ w
        port_vol = np.sqrt(port_var)
        marginal = cov @ w
        rc = w * marginal / port_vol
        rc_pct = rc / rc.sum()
        return np.sum((rc_pct - target_rc) ** 2)

    vols = np.sqrt(np.diag(cov))
    w0 = (1.0 / vols) / (1.0 / vols).sum()
    constraints = [{"type": "eq", "fun": lambda w: np.sum(w) - 1.0}]
    bounds = [(0.01, 0.50)] * n

    result = minimize(objective, w0, method="SLSQP", constraints=constraints,
                      bounds=bounds, options={"ftol": 1e-12, "maxiter": 1000})
    return result.x if result.success else w0


def risk_parity_by_class(weights, cov):
    """Compute risk contribution grouped by asset class."""
    rc = risk_contribution(weights, cov)
    class_rc = {}
    for i, asset in enumerate(ASSETS):
        cls = ASSET_CLASSES[asset]
        class_rc[cls] = class_rc.get(cls, 0.0) + rc[i]
    return class_rc


# Optimize
weights = risk_parity_weights(cov_annual)
risk_contribs = risk_contribution(weights, cov_annual)
class_rc = risk_parity_by_class(weights, cov_annual)

print("Risk Parity Portfolio Allocation:")
print(f"{'Asset':<20} {'Weight':>10} {'Risk Contrib':>14} {'Class':>10}")
print("-" * 56)
for i, asset in enumerate(ASSETS):
    print(f"{asset:<20} {weights[i]:>10.2%} {risk_contribs[i]:>14.2%} {ASSET_CLASSES[asset]:>10}")

print(f"\nRisk Contribution by Asset Class:")
for cls, rc_val in class_rc.items():
    print(f"  {cls:<12} {rc_val:.2%}")

## 6. Risk Metrics

Comprehensive risk and performance metrics computed across all 10,000 simulations:

- **VaR / CVaR**: Value-at-Risk and Conditional VaR (Expected Shortfall)
- **Maximum Drawdown**: Largest peak-to-trough decline
- **Sharpe Ratio**: $(\bar{r} - r_f) / \sigma$
- **Sortino Ratio**: Uses downside deviation only
- **Calmar Ratio**: Annualized return / max drawdown
- **Skewness / Kurtosis**: Distribution shape characteristics

In [ ]:
def maximum_drawdown(wealth_path):
    """Compute max drawdown from a 1D wealth path."""
    peak = np.maximum.accumulate(wealth_path)
    drawdown = (peak - wealth_path) / peak
    return float(np.max(drawdown))


def compute_all_metrics(portfolio_returns, rf=RISK_FREE_RATE):
    """Compute comprehensive risk/performance metrics."""
    n_sims, n_periods = portfolio_returns.shape
    daily_rf = rf / TRADING_DAYS_PER_YEAR

    total_return = np.prod(1.0 + portfolio_returns, axis=1)
    n_years = n_periods / TRADING_DAYS_PER_YEAR
    annualized_return = total_return ** (1.0 / n_years) - 1.0

    daily_vol = np.std(portfolio_returns, axis=1, ddof=1)
    annualized_vol = daily_vol * np.sqrt(TRADING_DAYS_PER_YEAR)

    sharpe = (annualized_return - rf) / annualized_vol

    excess = portfolio_returns - daily_rf
    downside = np.where(excess < 0, excess, 0.0)
    downside_vol = np.sqrt(np.mean(downside**2, axis=1)) * np.sqrt(TRADING_DAYS_PER_YEAR)
    sortino = (annualized_return - rf) / np.where(downside_vol > 0, downside_vol, np.nan)

    wealth = np.cumprod(1.0 + portfolio_returns, axis=1)
    max_dd = np.array([maximum_drawdown(wealth[i]) for i in range(n_sims)])

    calmar = annualized_return / np.where(max_dd > 0, max_dd, np.nan)

    all_returns = portfolio_returns.flatten()
    var_95 = np.percentile(all_returns, 5)
    var_99 = np.percentile(all_returns, 1)
    cvar_95 = all_returns[all_returns <= var_95].mean()
    cvar_99 = all_returns[all_returns <= var_99].mean()

    skew_vals = stats.skew(portfolio_returns, axis=1)
    kurt_vals = stats.kurtosis(portfolio_returns, axis=1)

    return {
        "annualized_return": annualized_return, "annualized_vol": annualized_vol,
        "sharpe": sharpe, "sortino": sortino, "max_drawdown": max_dd, "calmar": calmar,
        "skewness": skew_vals, "kurtosis": kurt_vals,
        "var_95": var_95, "var_99": var_99, "cvar_95": cvar_95, "cvar_99": cvar_99,
        "wealth": wealth,
    }


def summarize_metrics(metrics):
    """Summarize distributions into median [5th, 95th] percentiles."""
    summary = {}
    for key in ["annualized_return", "annualized_vol", "sharpe", "sortino",
                "max_drawdown", "calmar", "skewness", "kurtosis"]:
        vals = metrics[key]
        vals_clean = vals[np.isfinite(vals)]
        if len(vals_clean) > 0:
            summary[key] = {"median": float(np.median(vals_clean)),
                           "p5": float(np.percentile(vals_clean, 5)),
                           "p95": float(np.percentile(vals_clean, 95))}
    for key in ["var_95", "var_99", "cvar_95", "cvar_99"]:
        summary[key] = float(metrics[key])
    return summary


# Compute portfolio returns and metrics
portfolio_returns = np.einsum("ijk,k->ij", results["returns"], weights)
metrics = compute_all_metrics(portfolio_returns)
summary = summarize_metrics(metrics)

# Display
def fmt(v, pct=True):
    return f"{v:.2%}" if pct else f"{v:.2f}"

print(f"Portfolio Risk Metrics ({N_SIMULATIONS:,} simulations)")
print(f"{'Metric':<28} {'Median':>10} {'5th Pct':>10} {'95th Pct':>10}")
print("-" * 60)
for key, label, pct in [
    ("annualized_return", "Annualized Return", True),
    ("annualized_vol", "Annualized Volatility", True),
    ("sharpe", "Sharpe Ratio", False),
    ("sortino", "Sortino Ratio", False),
    ("max_drawdown", "Maximum Drawdown", True),
    ("calmar", "Calmar Ratio", False),
    ("skewness", "Skewness", False),
    ("kurtosis", "Excess Kurtosis", False),
]:
    s = summary[key]
    print(f"{label:<28} {fmt(s['median'], pct):>10} {fmt(s['p5'], pct):>10} {fmt(s['p95'], pct):>10}")

print(f"\nDaily VaR (95%):   {summary['var_95']:.4%}")
print(f"Daily VaR (99%):   {summary['var_99']:.4%}")
print(f"Daily CVaR (95%):  {summary['cvar_95']:.4%}")
print(f"Daily CVaR (99%):  {summary['cvar_99']:.4%}")

## 7. Visualizations

### 7.1 Simulation Paths
Fan chart showing 100 sample portfolio wealth paths with percentile bands across the 10-year horizon.

In [ ]:
# Color palette
COLORS = {
    "bull": "#00c853", "bear": "#ff6d00", "crisis": "#d50000",
    "primary": "#42a5f5", "secondary": "#ab47bc", "accent": "#26c6da", "gold": "#ffd600",
}
ASSET_COLORS = sns.color_palette("husl", len(ASSETS))

# Simulation paths
wealth = metrics["wealth"]
n_sims, n_periods = wealth.shape
t = np.arange(n_periods) / TRADING_DAYS_PER_YEAR

fig, ax = plt.subplots(figsize=(14, 7))

sample_rng = np.random.default_rng(0)
sample_idx = sample_rng.choice(n_sims, size=100, replace=False)
for i in sample_idx:
    ax.plot(t, wealth[i], alpha=0.05, color=COLORS["primary"], linewidth=0.5)

p5 = np.percentile(wealth, 5, axis=0)
p25 = np.percentile(wealth, 25, axis=0)
p50 = np.median(wealth, axis=0)
p75 = np.percentile(wealth, 75, axis=0)
p95 = np.percentile(wealth, 95, axis=0)

ax.fill_between(t, p5, p95, alpha=0.15, color=COLORS["primary"], label="5th-95th pct")
ax.fill_between(t, p25, p75, alpha=0.25, color=COLORS["primary"], label="25th-75th pct")
ax.plot(t, p50, color=COLORS["gold"], linewidth=2, label="Median")

ax.set_xlabel("Years", fontsize=12)
ax.set_ylabel("Portfolio Value ($1 initial)", fontsize=12)
ax.set_title("Monte Carlo Simulation: 10,000 Portfolio Paths", fontsize=14, fontweight="bold")
ax.legend(loc="upper left", fontsize=10)
ax.set_xlim(0, t[-1])
ax.grid(alpha=0.2)
plt.tight_layout()
plt.show()

### 7.2 Return Distribution & Q-Q Plot
The histogram shows the distribution of annualized returns with VaR thresholds marked. The Q-Q plot reveals deviation from normality — fat tails from the Student-t innovations are visible at the extremes.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ann_ret = metrics["annualized_return"]

ax1.hist(ann_ret, bins=80, density=True, alpha=0.7, color=COLORS["primary"], edgecolor="none", label="Simulated")
mu_fit, sigma_fit = ann_ret.mean(), ann_ret.std()
x = np.linspace(ann_ret.min(), ann_ret.max(), 200)
ax1.plot(x, stats.norm.pdf(x, mu_fit, sigma_fit), color=COLORS["gold"],
         linewidth=2, label=f"Normal fit (\u03bc={mu_fit:.2%}, \u03c3={sigma_fit:.2%})")
var_5 = np.percentile(ann_ret, 5)
var_1 = np.percentile(ann_ret, 1)
ax1.axvline(var_5, color=COLORS["bear"], linestyle="--", linewidth=1.5, label=f"5th pct: {var_5:.2%}")
ax1.axvline(var_1, color=COLORS["crisis"], linestyle="--", linewidth=1.5, label=f"1st pct: {var_1:.2%}")
ax1.set_xlabel("Annualized Return", fontsize=11)
ax1.set_ylabel("Density", fontsize=11)
ax1.set_title("Distribution of Annualized Returns", fontsize=13, fontweight="bold")
ax1.legend(fontsize=9)
ax1.xaxis.set_major_formatter(mticker.PercentFormatter(1.0))
ax1.grid(alpha=0.2)

stats.probplot(ann_ret, dist="norm", plot=ax2)
ax2.set_title("Q-Q Plot vs. Normal", fontsize=13, fontweight="bold")
ax2.get_lines()[0].set_color(COLORS["primary"])
ax2.get_lines()[0].set_markersize(2)
ax2.get_lines()[1].set_color(COLORS["crisis"])
ax2.grid(alpha=0.2)

plt.tight_layout()
plt.show()

### 7.3 Regime Distribution Over Time
Shows the fraction of simulations in each market regime at every point in time. The Markov chain converges to its stationary distribution, demonstrating ergodicity.

In [ ]:
regimes = results["regimes"]
n_sims_r, n_periods_r = regimes.shape
t_r = np.arange(n_periods_r) / TRADING_DAYS_PER_YEAR

fractions = np.zeros((3, n_periods_r))
for r in range(3):
    fractions[r] = (regimes == r).mean(axis=0)

fig, ax = plt.subplots(figsize=(14, 5))
colors = [COLORS["bull"], COLORS["bear"], COLORS["crisis"]]
labels = ["Bull", "Bear", "Crisis"]
ax.stackplot(t_r, fractions, colors=colors, labels=labels, alpha=0.85)
ax.set_xlabel("Years", fontsize=12)
ax.set_ylabel("Fraction of Simulations", fontsize=12)
ax.set_title("Regime Distribution Over Time (Markov Chain)", fontsize=14, fontweight="bold")
ax.legend(loc="upper right", fontsize=10)
ax.set_xlim(0, t_r[-1])
ax.set_ylim(0, 1)
ax.grid(alpha=0.2)
plt.tight_layout()
plt.show()

### 7.4 Risk Parity Allocation
Capital weights (left) vs. risk contribution (right). Despite unequal capital allocation, each asset contributes exactly 12.5% of total portfolio risk — the defining property of risk parity.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
x = np.arange(len(ASSETS))
short_names = [a.replace("_", "\n") for a in ASSETS]

ax1.bar(x, weights, color=ASSET_COLORS, edgecolor="white", linewidth=0.5)
ax1.set_xticks(x)
ax1.set_xticklabels(short_names, fontsize=8)
ax1.set_ylabel("Weight", fontsize=11)
ax1.set_title("Capital Allocation", fontsize=13, fontweight="bold")
ax1.yaxis.set_major_formatter(mticker.PercentFormatter(1.0))
ax1.grid(alpha=0.2, axis="y")

ax2.bar(x, risk_contribs, color=ASSET_COLORS, edgecolor="white", linewidth=0.5)
ax2.set_xticks(x)
ax2.set_xticklabels(short_names, fontsize=8)
ax2.set_ylabel("Risk Contribution", fontsize=11)
ax2.set_title("Risk Contribution (Target: Equal)", fontsize=13, fontweight="bold")
ax2.yaxis.set_major_formatter(mticker.PercentFormatter(1.0))
ax2.axhline(1.0 / len(ASSETS), color=COLORS["gold"], linestyle="--",
            linewidth=1.5, label=f"Target: {1/len(ASSETS):.1%}")
ax2.legend(fontsize=10)
ax2.grid(alpha=0.2, axis="y")

plt.tight_layout()
plt.show()

### 7.5 Efficient Frontier
The classical Markowitz efficient frontier with individual assets, the risk parity portfolio, and the equal-weight portfolio plotted for comparison.

In [ ]:
def efficient_frontier(mu, cov, n_points=50):
    """Compute the efficient frontier via quadratic optimization."""
    n = len(mu)
    def portfolio_vol(w):
        return np.sqrt(w @ cov @ w)

    min_ret, max_ret = mu.min(), mu.max()
    target_returns = np.linspace(min_ret, max_ret, n_points)
    frontier_vols, frontier_rets, frontier_weights = [], [], []
    w0 = np.ones(n) / n
    bounds = [(0.0, 1.0)] * n

    for target in target_returns:
        constraints = [
            {"type": "eq", "fun": lambda w: np.sum(w) - 1.0},
            {"type": "eq", "fun": lambda w, t=target: w @ mu - t},
        ]
        result = minimize(portfolio_vol, w0, method="SLSQP", constraints=constraints,
                          bounds=bounds, options={"ftol": 1e-10, "maxiter": 500})
        if result.success:
            frontier_vols.append(portfolio_vol(result.x))
            frontier_rets.append(target)
            frontier_weights.append(result.x.copy())

    return {"returns": np.array(frontier_rets), "vols": np.array(frontier_vols),
            "weights": np.array(frontier_weights)}

frontier = efficient_frontier(mu_annual, cov_annual, n_points=50)

fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(frontier["vols"], frontier["returns"], color=COLORS["primary"], linewidth=2, label="Efficient Frontier")

asset_vols = np.sqrt(np.diag(cov_annual))
for i, (name, vol, ret) in enumerate(zip(ASSETS, asset_vols, mu_annual)):
    ax.scatter(vol, ret, color=ASSET_COLORS[i], s=80, zorder=5, edgecolors="white")
    ax.annotate(name.replace("_", " "), (vol, ret), textcoords="offset points",
                xytext=(8, 4), fontsize=8, color="white")

rp_vol = np.sqrt(weights @ cov_annual @ weights)
rp_ret = weights @ mu_annual
ax.scatter(rp_vol, rp_ret, color=COLORS["gold"], s=200, marker="*",
           zorder=10, edgecolors="white", linewidth=1, label="Risk Parity")

ew = np.ones(len(mu_annual)) / len(mu_annual)
ew_vol = np.sqrt(ew @ cov_annual @ ew)
ew_ret = ew @ mu_annual
ax.scatter(ew_vol, ew_ret, color=COLORS["accent"], s=100, marker="D",
           zorder=10, edgecolors="white", linewidth=1, label="Equal Weight")

ax.set_xlabel("Annualized Volatility", fontsize=12)
ax.set_ylabel("Annualized Return", fontsize=12)
ax.set_title("Efficient Frontier with Risk Parity Portfolio", fontsize=14, fontweight="bold")
ax.xaxis.set_major_formatter(mticker.PercentFormatter(1.0))
ax.yaxis.set_major_formatter(mticker.PercentFormatter(1.0))
ax.legend(fontsize=10, loc="upper left")
ax.grid(alpha=0.2)
plt.tight_layout()
plt.show()

## 8. Macro Factor Decomposition

We decompose each asset's return variance into contributions from three systematic macro factors:
- **Growth**: Captures economic expansion/contraction sensitivity
- **Inflation**: Captures real asset vs. nominal asset dynamics  
- **Rates**: Captures interest rate sensitivity (duration)

The decomposition uses OLS projection: $F = (B^T B)^{-1} B^T R^T$

In [ ]:
def decompose_returns(returns):
    """Decompose asset returns into macro factor components via OLS projection."""
    B = FACTOR_LOADINGS
    R = returns.mean(axis=0) if returns.ndim == 3 else returns

    BtB_inv = np.linalg.inv(B.T @ B)
    factor_returns = (BtB_inv @ B.T @ R.T).T
    explained = factor_returns @ B.T
    residual = R - explained

    n_assets, n_factors = B.shape
    var_decomp = np.zeros((n_assets, n_factors + 1))
    for f in range(n_factors):
        factor_component = np.outer(factor_returns[:, f], B[:, f])
        var_decomp[:, f] = np.var(factor_component, axis=0)
    var_decomp[:, -1] = np.var(residual, axis=0)

    row_sums = var_decomp.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1.0
    return var_decomp / row_sums


var_decomp = decompose_returns(results["returns"])

# Plot
factor_labels = ["Growth", "Inflation", "Rates", "Idiosyncratic"]
factor_colors = [COLORS["primary"], COLORS["bear"], COLORS["accent"], "#757575"]

fig, ax = plt.subplots(figsize=(14, 6))
x = np.arange(len(ASSETS))
short_names = [a.replace("_", "\n") for a in ASSETS]

bottom = np.zeros(len(ASSETS))
for f in range(4):
    ax.bar(x, var_decomp[:, f], bottom=bottom, color=factor_colors[f],
           label=factor_labels[f], edgecolor="white", linewidth=0.5)
    bottom += var_decomp[:, f]

ax.set_xticks(x)
ax.set_xticklabels(short_names, fontsize=9)
ax.set_ylabel("Fraction of Variance", fontsize=11)
ax.set_title("Macro Factor Variance Decomposition", fontsize=14, fontweight="bold")
ax.legend(fontsize=10)
ax.set_ylim(0, 1.05)
ax.yaxis.set_major_formatter(mticker.PercentFormatter(1.0))
ax.grid(alpha=0.2, axis="y")
plt.tight_layout()
plt.show()

## 9. Stress Testing

We apply four historical/hypothetical stress scenarios to the risk parity portfolio. Each scenario defines asset-level shocks calibrated to observed market behavior during:

| Scenario | Period | Key Driver |
|----------|--------|------------|
| **GFC 2008** | Sep 2008 - Mar 2009 | Credit crisis, equity crash, flight to quality |
| **COVID 2020** | Feb - Mar 2020 | Pandemic shock, indiscriminate selling |
| **Rate Shock** | Hypothetical | Rapid rate hikes, bond/equity correlation flip |
| **Stagflation** | Hypothetical | Rising inflation + falling growth |

In [ ]:
def run_stress_tests(weights, scenarios=STRESS_SCENARIOS):
    """Run all stress scenarios against the portfolio."""
    rows = []
    for name, shocks in scenarios.items():
        shock_vec = np.array([shocks[a] for a in ASSETS])
        portfolio_impact = weights @ shock_vec
        contributions = weights * shock_vec
        row = {"Scenario": name, "Portfolio_Impact": portfolio_impact}
        for i, asset in enumerate(ASSETS):
            row[asset] = contributions[i]
        rows.append(row)
    return pd.DataFrame(rows).set_index("Scenario")


stress_df = run_stress_tests(weights)

# Display table
print("Stress Test Results:")
print(f"{'Scenario':<20} {'Portfolio Impact':>18}")
print("-" * 40)
for scenario in stress_df.index:
    impact = stress_df.loc[scenario, "Portfolio_Impact"]
    print(f"{scenario:<20} {impact:>18.2%}")

# Waterfall chart
fig, ax = plt.subplots(figsize=(12, 6))
scenarios = stress_df.index.tolist()
impacts = stress_df["Portfolio_Impact"].values
bar_colors = [COLORS["crisis"] if v < 0 else COLORS["bull"] for v in impacts]

bars = ax.barh(scenarios, impacts, color=bar_colors, edgecolor="white", height=0.5)
for bar, val in zip(bars, impacts):
    x_pos = val - 0.005 if val < 0 else val + 0.005
    ha = "right" if val < 0 else "left"
    ax.text(x_pos, bar.get_y() + bar.get_height() / 2,
            f"{val:.1%}", ha=ha, va="center", fontsize=11, fontweight="bold")

ax.axvline(0, color="white", linewidth=0.5)
ax.set_xlabel("Portfolio Impact", fontsize=12)
ax.set_title("Stress Test Results", fontsize=14, fontweight="bold")
ax.xaxis.set_major_formatter(mticker.PercentFormatter(1.0))
ax.grid(alpha=0.2, axis="x")
ax.invert_yaxis()
plt.tight_layout()
plt.show()

### 9.1 Drawdown Distribution
Distribution of maximum drawdowns across all 10,000 simulated paths over the 10-year horizon.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

max_dd = metrics["max_drawdown"]
ax.hist(max_dd, bins=80, density=True, alpha=0.7, color=COLORS["crisis"], edgecolor="none")

p50_dd = np.median(max_dd)
p95_dd = np.percentile(max_dd, 95)
p99_dd = np.percentile(max_dd, 99)

ax.axvline(p50_dd, color=COLORS["gold"], linestyle="--", linewidth=1.5, label=f"Median: {p50_dd:.1%}")
ax.axvline(p95_dd, color=COLORS["bear"], linestyle="--", linewidth=1.5, label=f"95th pct: {p95_dd:.1%}")
ax.axvline(p99_dd, color="white", linestyle="--", linewidth=1.5, label=f"99th pct: {p99_dd:.1%}")

ax.set_xlabel("Maximum Drawdown", fontsize=12)
ax.set_ylabel("Density", fontsize=12)
ax.set_title("Distribution of Maximum Drawdowns (10-Year Horizon)", fontsize=14, fontweight="bold")
ax.xaxis.set_major_formatter(mticker.PercentFormatter(1.0))
ax.legend(fontsize=10)
ax.grid(alpha=0.2)
plt.tight_layout()
plt.show()

## 10. Summary

### Key Findings

1. **Risk parity achieves perfect equal risk contribution** — each of the 8 assets contributes exactly 12.5% of portfolio risk, and each asset class contributes 25%

2. **Fat tails are present** — excess kurtosis significantly above 0 confirms that the Student-t innovations produce realistic heavy-tailed return distributions

3. **Regime switching creates path-dependent dynamics** — crisis regimes simultaneously increase volatility, spike correlations (destroying diversification), and produce negative expected returns

4. **The portfolio is resilient to equity-driven shocks** (GFC, COVID) thanks to bond and gold hedges, but vulnerable to rate shocks that hit bonds and equities simultaneously

5. **Risk parity sits near the minimum-variance point** on the efficient frontier, prioritizing risk management over return maximization

### Methodology

| Component | Approach |
|-----------|----------|
| Return generation | Cholesky decomposition + Student-t innovations |
| Regime dynamics | 3-state Markov chain (Bull/Bear/Crisis) |
| Allocation | Risk parity via SLSQP optimization |
| Factor model | OLS projection onto Growth/Inflation/Rates |
| Risk metrics | Historical VaR/CVaR, drawdown, Sharpe/Sortino/Calmar |
| Stress testing | Deterministic scenario shocks |

In [ ]:
# Final metrics table
fig, ax = plt.subplots(figsize=(10, 6))
ax.axis("off")

rows = []
row_labels = []

for key, label, pct in [
    ("annualized_return", "Annualized Return", True),
    ("annualized_vol", "Annualized Volatility", True),
    ("sharpe", "Sharpe Ratio", False),
    ("sortino", "Sortino Ratio", False),
    ("max_drawdown", "Maximum Drawdown", True),
    ("calmar", "Calmar Ratio", False),
    ("skewness", "Skewness", False),
    ("kurtosis", "Excess Kurtosis", False),
]:
    s = summary[key]
    rows.append([fmt(s["median"], pct), fmt(s["p5"], pct), fmt(s["p95"], pct)])
    row_labels.append(label)

for key, label in [("var_95", "Daily VaR (95%)"), ("var_99", "Daily VaR (99%)"),
                   ("cvar_95", "Daily CVaR (95%)"), ("cvar_99", "Daily CVaR (99%)")]:
    rows.append([f"{summary[key]:.4%}", "\u2014", "\u2014"])
    row_labels.append(label)

col_labels = ["Median", "5th Pct", "95th Pct"]
table = ax.table(cellText=rows, rowLabels=row_labels, colLabels=col_labels,
                 cellLoc="center", loc="center")
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.6)

for (row, col), cell in table.get_celld().items():
    cell.set_edgecolor("#555555")
    if row == 0:
        cell.set_facecolor("#333333")
        cell.set_text_props(fontweight="bold", color="white")
    elif col == -1:
        cell.set_facecolor("#2a2a2a")
        cell.set_text_props(fontweight="bold", color="white")
    else:
        cell.set_facecolor("#1a1a1a")
        cell.set_text_props(color="white")

ax.set_title("Portfolio Risk Metrics Summary (10,000 Simulations)",
             fontsize=14, fontweight="bold", pad=20)
plt.tight_layout()
plt.show()